## 概要

- image.py: 画像のみで学習。
- stage1.py: 256px解像度の動画で学習。
- stage2.py: 768px解像度の動画で学習（シーケンス並列化を使用、デフォルトは4）。
- stage1_i2v.py: 256px解像度でT2V（テキストから動画）とI2V（画像から動画）を学習。
- stage2_i2v.py: 768px解像度でT2VとI2Vを学習。

## 環境構築

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%cd /workspaces/open-sora/Open-Sora
%pip install -e . --no-deps

In [ ]:
import logging
import os
import platform
import subprocess

if os.path.exists("debug.log"):
    os.remove("debug.log")

def custom_format(record):
    match record.levelno:
        case logging.DEBUG:
            level = "🟦"
        case logging.INFO:
            level = "🟩"
        case logging.WARNING:
            level = "🟨"
        case logging.ERROR:
            level = "🟥"
        case logging.CRITICAL:
            level = "🛑"
    return f"{level} {record.getMessage()}"

logger_ = logging.getLogger()

for handler in logger_.handlers:
    logger_.removeHandler(handler)

formatter = logging.Formatter()
formatter.format = custom_format

file_handler = logging.FileHandler("debug.log")
file_handler.setFormatter(formatter)
logger_.addHandler(file_handler)

stream_handler = logging.StreamHandler()
stream_handler.setFormatter(formatter)
logger_.addHandler(stream_handler)
logger_.setLevel(logging.DEBUG)

PYTHON_VERSION = platform.python_version()
logger_.info(f"Python {PYTHON_VERSION}")

NVIDIA_SMI = subprocess.run("nvidia-smi", capture_output=True, text=True).stdout
logger_.info(f"{NVIDIA_SMI}")

In [ ]:
# %pip install torch==2.6.0 torchvision==0.21.0 torchaudio==2.6.0 --index-url https://download.pytorch.org/whl/cu124
%pip install torch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1 --index-url https://download.pytorch.org/whl/cu124

In [ ]:
%pip install \
    accelerate \
    av \
    colossalai \
    ftfy \
    liger-kernel \
    omegaconf \
    mmengine \
    openai \
    pandas \
    pandarallel \
    pyarrow \
    tensorboard \
    wandb \
    --extra-index-url https://download.pytorch.org/whl/cu124

In [ ]:
%pip install flash-attn --no-build-isolation

### ダウンロード

In [ ]:
# 269.53GBの動画データセットをダウンロード
# https://modelscope.cn/datasets/hpcai-tech/open-sora-pexels-45k

if False:
    !apt update && apt install git-lfs
    !git-lfs install
    !git clone "https://www.modelscope.cn/datasets/hpcai-tech/open-sora-pexels-45k.git"
    !cd open-sora-pexels-45k && \
        cat tar/pexels_45k.tar.* > pexels_45k.tar && \
        mkdir ../datasets && \
        mv pexels_45k ../datasets

In [ ]:
# 数件の動画を直接ダウンロード

if False:
    from multiprocessing import Pool
    import os
    import pandas as pd
    import re
    import subprocess

    os.makedirs("pexel_10", exist_ok=True)
    necessary_csv_path = "/workspaces/open-sora/pexels_10/pexels_10_necessary.csv"
    df = pd.read_csv(necessary_csv_path)

    video_urls = []

    for path in df["path"]:
        match = re.search(r"/(\d+)_", path)
        video_id = match.group(1)
        video_url = f"https://www.pexels.com/download/video/{video_id}"
        df.loc[df["path"] == path, "url"] = video_url

    def download_video(args):
        url, save_path = args

        # 保存先のディレクトリを作成（存在しない場合）
        directory = os.path.dirname(save_path)
        if directory:
            os.makedirs(directory, exist_ok=True)
        
        # ファイルが既に存在しない場合のみダウンロードを実行
        if not os.path.exists(save_path):
            try:
                # -O: 出力ファイル名を指定
                # -q: ログ出力を抑制（進捗が見たい場合は外してください）
                command = ["wget", "-q", "-O", save_path, url]
                subprocess.run(command, check=True)
            except subprocess.CalledProcessError:
                print(f"Failed to download: {url}")

    # URLと保存パスのペアを作成
    tasks = list(zip(df["url"], df["path"]))

    with Pool(os.cpu_count()) as p:
        p.map(download_video, tasks)

In [ ]:
# 事前学習済みVAEのダウンロード

os.makedirs("ckpts", exist_ok=True)

if not os.path.exists("ckpts/hunyuan_vae.safetensors"):
    !huggingface-cli download hpcai-tech/Open-Sora-v2 hunyuan_vae.safetensors --local-dir ckpts --local-dir-use-symlinks False
else:
    logger_.info(f"既にVAEをダウンロード済み")

## 訓練

In [ ]:
# !torchrun --nproc_per_node=1 scripts/diffusion/train.py configs/diffusion/train/stage1.py --dataset.data-path /workspaces/open-sora/pexels_45k/pexels_45k_necessary.csv